In [17]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [19]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [21]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [23]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [62]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

scaled = StandardScaler().fit_transform(df_market_data)

In [98]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(scaled, columns=df_market_data.columns, index=df_market_data.index)
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


### Find the Best Value for k Using the Original Scaled DataFrame.

In [107]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

# Create an empty list to store the inertia values

inertia = []

# Create a for loop to compute the inertia with each possible value of k

for i in k:

# Inside the loop:
# 1. Create a KMeans model using the loop counter for the 

    k_model = KMeans(n_clusters=i, random_state=0)
    
# 2. Fit the model to the data using `df_market_data_scaled`

    k_model.fit(df_market_data_scaled)
    
# 3. Append the model.inertia_ to the inertia list

    inertia.append(k_model.inertia_)


In [109]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow

,k,inertia
0,1,287.000000
1,2,198.571818
2,3,145.635748
3,4,79.022435
4,5,66.217780
5,6,59.139070
6,7,48.406314
7,8,38.993588
8,9,34.400595
9,10,28.573603


In [296]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.

df_elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
df_elbow_plot

:Curve   [k]   (inertia)


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4 

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [197]:
# Initialize the K-Means model using the best value for k

model = KMeans(n_clusters=4, random_state=0)

In [199]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=0)

In [201]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_4 = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
k_4

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [203]:
# Create a copy of the scaled DataFrame
df_market_data_scaled_copy= df_market_data_scaled.copy()

In [205]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters

df_market_data_scaled_copy["Predicted"] = k_4
# Display the copy of the scaled DataFrame
df_market_data_scaled_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Predicted
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


### Reset index to hover coin_id in the scatter plot. 

In [208]:
df_market_data_scaled_copy.reset_index(inplace=True)

In [310]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

scatter_plot = df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="Predicted",
    hover_cols = "coin_id"
)
scatter_plot

:NdOverlay   [Predicted]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### K=4 was not the good choice. So, trying with k=3

In [215]:
# Initialize the K-Means model using the best value for k

model = KMeans(n_clusters=3, random_state=0)
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_3 = model.predict(df_market_data_scaled)
# Create a copy of the scaled DataFrame
df_market_data_scaled_copy2= df_market_data_scaled.copy()
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_scaled_copy2["Predicted"] = k_3

df_market_data_scaled_copy2.reset_index(inplace=True)

df_market_data_scaled_copy2.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="Predicted",
    hover_cols = "coin_id"
)


:NdOverlay   [Predicted]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### k=3 is better than k=4.

---

### Optimize Clusters with Principal Component Analysis.

In [225]:
# Create a PCA model instance and set `n_components=3`.

pca = PCA(n_components=3)

In [232]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_transformed = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
pca_transformed


array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527],
       [-0.51653377,  1.38837748,  0.80407131],
       [-0.45071134,  0.51769912,  2.84614316],
       [-0.34559977,  0.72943939,  1.47801284],
       [-0.64946792,  0.43216514,  0.60030286],
       [-0.75901394, -0.20119979, -0.21765292],
       [-0.24819846, -1.37625159, -1.46202571],
       [-0.43840762, -0.17533654, -0.6633884 ],
       [-0.69342533, -0.47381462, -0.52759693],
       [ 0.06049915,  2.90940385,  1.49857131],
       [-0.39335243, -0.10819197, -0.01275608],
       [-0.79617564, -0.49440875,  1.08281169],
       [ 0.06407452, -1.26982514, -1.09882928],
       [-0.48901506, -0.73271912, -0.06254323],
       [-0.3062723 ,  0.70341515,  1.71422359],
       [-0.51352775, -0.14280239, -0.65656583],
       [-0.36212044, -0.98691441, -0.728

In [236]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.

pca.explained_variance_ratio_


array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89%

In [260]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(pca_transformed)

# Copy the crypto names from the original scaled DataFrame

df_market_data_pca["coin_id"] = df_market_data_scaled.index

# Set the coin_id column as index

df_market_data_pca.set_index("coin_id", inplace=True)

# Display the scaled PCA DataFrame
df_market_data_pca.head()

,0,1,2
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [266]:
# Rename columns with PC1, PC2, and PC3

df_market_data_pca = df_market_data_pca.rename(columns = {0:"PC1", 1:"PC2", 2:"PC3"})

df_market_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [21]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [253]:
# Create an empty list to store the inertia values

inertia = []

# Create a for loop to compute the inertia with each possible value of k

for i in k:

# Inside the loop:
# 1. Create a KMeans model using the loop counter for the 

    k_model = KMeans(n_clusters=i, random_state=0)
    
# 2. Fit the model to the data using `df_market_data_scaled`

    k_model.fit(df_market_data_pca)
    
# 3. Append the model.inertia_ to the inertia list

    inertia.append(k_model.inertia_)


In [255]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow_pca = pd.DataFrame(elbow_data_pca)

# Review the DataFrame
df_elbow_pca


,k,inertia
0,1,256.874086
1,2,168.811896
2,3,115.918077
3,4,49.665497
4,5,41.357314
5,6,30.796642
6,7,25.915457
7,8,21.890936
8,9,17.720699
9,10,13.838699


In [298]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow_pca_plot = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

df_elbow_pca_plot


:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The suggestion by graph was same (4). 

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [269]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [275]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

KMeans(n_clusters=4, random_state=0)

In [277]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
k_4_pca = model.predict(df_market_data_pca)


# Print the resulting array of cluster values.
k_4_pca

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [279]:
# Create a copy of the scaled PCA DataFrame
df_market_data_pca_copy= df_market_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_market_data_pca_copy["Predicted"] = k_4_pca

# Display the copy of the scaled PCA DataFrame
df_market_data_pca_copy.head()

,PC1,PC2,PC3,Predicted
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [ ]:
df_market_data_pca_copy.reset_index(inplace=True)

In [328]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

pca_scatter_plot1 = df_market_data_pca_copy.hvplot.scatter(
    x="PC1", 
    y="PC2", 
    by="Predicted",
    hover_cols = "coin_id"
)
pca_scatter_plot1

:NdOverlay   [Predicted]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

#### Composite with * symbol

In [346]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!

combined_kmeans_plot = (df_elbow_plot * df_elbow_pca_plot).opts(
    title='ElbowCurve and PCA_Elbow_Curve',
    legend_position='top_right',
    show_legend=True #This is not showing (needed to update 
)

# Display the combined plot
combined_kmeans_plot


:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

#### Composite with + symbol

In [353]:
combined_kmeans_plot2 = ((df_elbow_plot + df_elbow_pca_plot).cols(1)).opts(
    title='ElbowCurve and PCA_Elbow_Curve',
    legend_position='top_right',
    show_legends=True, #This is not showing (needed to update 
    )

# Display the combined plot
combined_kmeans_plot2

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

#### Composite with * symbol (Clusters)

In [326]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

combined_cluster_plot = (scatter_plot * pca_scatter_plot1).opts(
    title='Scatter_plot and PCA_Scatter_pPlot',
    
    legend_position='top_right',
    show_legend=True
)

# Display the combined plot
combined_cluster_plot


:Overlay
   .NdOverlay.I  :NdOverlay   [Predicted]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Predicted]
      :Scatter   [PC1]   (PC2,coin_id)

#### Composite with + symbol (Clusters)

In [359]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

combined_cluster_plot2 = ((scatter_plot + pca_scatter_plot1).cols(1)).opts(
    title='Scatter_plot and PCA_Scatter_pPlot',
    legend_position='top_right',
    show_legends=True
)

# Display the combined plot
combined_cluster_plot2

:Layout
   .NdOverlay.I  :NdOverlay   [Predicted]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Predicted]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** It appears that the coin_segmentation information using the DataFrame with all of the factors yielded similar results that the PCA analysis.